<a href="https://colab.research.google.com/github/sadnyd/EdgarGraphLLM/blob/main/notebooks/investors_add.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dotenv langchain_community neo4j langchain langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.17 which is incompatible.


In [ ]:
# Common data processing
from google.colab import userdata
import json
import textwrap
import warnings
warnings.filterwarnings("ignore")
# LangChain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_google_genai import GoogleGenerativeAIEmbeddings  # Gemini Embeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_google_genai import ChatGoogleGenerativeAI        # Gemini Chat

In [ ]:
# Load from Colab userdata
NEO4J_URI = userdata.get('EDGAR_URI')
NEO4J_USERNAME = userdata.get('EDGAR_USERNAME')
NEO4J_PASSWORD = userdata.get('EDGAR_PASSWORD')
NEO4J_DATABASE = userdata.get('EDGAR_DATABASE')
GEMINI_API_KEY = userdata.get('gemini')
GEMINI_ENDPOINT = 'https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:embedText'

In [ ]:
# Global constants
VECTOR_INDEX_NAME = 'form_10k_chunks'
VECTOR_NODE_LABEL = 'Chunk'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY = 'textEmbedding'

In [ ]:
kg = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database=NEO4J_DATABASE
)

<ipython-input-6-15ba7034caff>:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  kg = Neo4jGraph(


In [ ]:
import csv

all_form13s = []

with open('./form13.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader: # each row will be a dictionary
      all_form13s.append(row)

In [ ]:
all_form13s[0:5]

[{'source': 'https://sec.gov/Archives/edgar/data/1000275/0001140361-23-039575.txt',
  'managerCik': '1000275',
  'managerAddress': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5',
  'managerName': 'Royal Bank of Canada',
  'reportCalendarOrQuarter': '2023-06-30',
  'cusip6': '64110D',
  'cusip': '64110D104',
  'companyName': 'NETAPP INC',
  'value': '64395000000.0',
  'shares': '842850'},
 {'source': 'https://sec.gov/Archives/edgar/data/1002784/0001387131-23-009542.txt',
  'managerCik': '1002784',
  'managerAddress': '1875 Lawrence Street, Suite 300, Denver, CO, 80202-1805',
  'managerName': 'SHELTON CAPITAL MANAGEMENT',
  'reportCalendarOrQuarter': '2023-06-30',
  'cusip6': '64110D',
  'cusip': '64110D104',
  'companyName': 'NETAPP INC',
  'value': '2989085000.0',
  'shares': '39124'},
 {'source': 'https://sec.gov/Archives/edgar/data/1007280/0001007280-23-000008.txt',
  'managerCik': '1007280',
  'managerAddress': '277 E TOWN ST, COLUMBUS, OH, 43215',
  'managerName': 'PUBLIC 

In [ ]:
len(all_form13s)

561

In [ ]:
# work with just the first form fow now
first_form13 = all_form13s[0]

cypher = """
MERGE (com:Company {cusip6: $cusip6})
  ON CREATE
    SET com.companyName = $companyName,
        com.cusip = $cusip
"""

kg.query(cypher, params={
    'cusip6':first_form13['cusip6'],
    'companyName':first_form13['companyName'],
    'cusip':first_form13['cusip']
})

[]

In [ ]:
cypher = """
MATCH (com:Company)
RETURN com LIMIT 1
"""

kg.query(cypher)

[{'com': {'cusip': '64110D104',
   'companyName': 'NETAPP INC',
   'cusip6': '64110D'}}]

In [ ]:
cypher = """
  MATCH (com:Company), (form:Form)
    WHERE com.cusip6 = form.cusip6
  RETURN com.companyName, form.names
"""

kg.query(cypher)

[{'com.companyName': 'NETAPP INC', 'form.names': ['Netapp Inc', 'NETAPP INC']}]

In [ ]:
cypher = """
  MATCH (com:Company), (form:Form)
    WHERE com.cusip6 = form.cusip6
  SET com.names = form.names
"""

kg.query(cypher)

[]

In [ ]:
kg.query("""
  MATCH (com:Company), (form:Form)
    WHERE com.cusip6 = form.cusip6
  MERGE (com)-[:FILED]->(form)
""")


[]

In [ ]:
cypher = """
  MERGE (mgr:Manager {managerCik: $managerParam.managerCik})
    ON CREATE
        SET mgr.managerName = $managerParam.managerName,
            mgr.managerAddress = $managerParam.managerAddress
"""

kg.query(cypher, params={'managerParam': first_form13})

[]

In [ ]:
kg.query("""
  MATCH (mgr:Manager)
  RETURN mgr LIMIT 1
""")

[{'mgr': {'managerCik': '1000275',
   'managerAddress': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5',
   'managerName': 'Royal Bank of Canada'}}]

In [ ]:
kg.query("""
CREATE CONSTRAINT unique_manager
  IF NOT EXISTS
  FOR (n:Manager)
  REQUIRE n.managerCik IS UNIQUE
""")

[]

In [ ]:
kg.query("""
CREATE FULLTEXT INDEX fullTextManagerNames
  IF NOT EXISTS
  FOR (mgr:Manager)
  ON EACH [mgr.managerName]
""")


[]

In [ ]:
kg.query("""
  CALL db.index.fulltext.queryNodes("fullTextManagerNames",
      "royal bank") YIELD node, score
  RETURN node.managerName, score
""")

[{'node.managerName': 'Royal Bank of Canada', 'score': 0.2615291476249695}]

In [ ]:
cypher = """
  MERGE (mgr:Manager {managerCik: $managerParam.managerCik})
    ON CREATE
        SET mgr.managerName = $managerParam.managerName,
            mgr.managerAddress = $managerParam.managerAddress
"""
# loop through all Form 13s
for form13 in all_form13s:
  kg.query(cypher, params={'managerParam': form13 })

In [ ]:
kg.query("""
    MATCH (mgr:Manager)
    RETURN count(mgr)
""")

[{'count(mgr)': 561}]

In [ ]:
cypher = """
  MATCH (mgr:Manager {managerCik: $investmentParam.managerCik}),
        (com:Company {cusip6: $investmentParam.cusip6})
  RETURN mgr.managerName, com.companyName, $investmentParam as investment
"""

kg.query(cypher, params={
    'investmentParam': first_form13
})

[{'mgr.managerName': 'Royal Bank of Canada',
  'com.companyName': 'NETAPP INC',
  'investment': {'shares': '842850',
   'source': 'https://sec.gov/Archives/edgar/data/1000275/0001140361-23-039575.txt',
   'managerName': 'Royal Bank of Canada',
   'managerAddress': 'ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5',
   'value': '64395000000.0',
   'cusip6': '64110D',
   'cusip': '64110D104',
   'reportCalendarOrQuarter': '2023-06-30',
   'companyName': 'NETAPP INC',
   'managerCik': '1000275'}}]

In [ ]:
cypher = """
MATCH (mgr:Manager {managerCik: $ownsParam.managerCik}),
        (com:Company {cusip6: $ownsParam.cusip6})
MERGE (mgr)-[owns:OWNS_STOCK_IN {
    reportCalendarOrQuarter: $ownsParam.reportCalendarOrQuarter
}]->(com)
ON CREATE
    SET owns.value  = toFloat($ownsParam.value),
        owns.shares = toInteger($ownsParam.shares)
RETURN mgr.managerName, owns.reportCalendarOrQuarter, com.companyName
"""

kg.query(cypher, params={ 'ownsParam': first_form13 })

[{'mgr.managerName': 'Royal Bank of Canada',
  'owns.reportCalendarOrQuarter': '2023-06-30',
  'com.companyName': 'NETAPP INC'}]

In [ ]:
kg.query("""
MATCH (mgr:Manager {managerCik: $ownsParam.managerCik})
-[owns:OWNS_STOCK_IN]->
        (com:Company {cusip6: $ownsParam.cusip6})
RETURN owns { .shares, .value }
""", params={ 'ownsParam': first_form13 })

[{'owns': {'shares': 842850, 'value': 64395000000.0}}]

In [ ]:
cypher = """
MATCH (mgr:Manager {managerCik: $ownsParam.managerCik}),
        (com:Company {cusip6: $ownsParam.cusip6})
MERGE (mgr)-[owns:OWNS_STOCK_IN {
    reportCalendarOrQuarter: $ownsParam.reportCalendarOrQuarter
    }]->(com)
  ON CREATE
    SET owns.value  = toFloat($ownsParam.value),
        owns.shares = toInteger($ownsParam.shares)
"""

#loop through all Form 13s
for form13 in all_form13s:
  kg.query(cypher, params={'ownsParam': form13 })

In [ ]:
cypher = """
  MATCH (:Manager)-[owns:OWNS_STOCK_IN]->(:Company)
  RETURN count(owns) as investments
"""

kg.query(cypher)

[{'investments': 561}]

In [ ]:
kg.refresh_schema()
print(textwrap.fill(kg.schema, 60))

Node properties: Chunk {chunkId: STRING, names: LIST,
formId: STRING, cik: STRING, cusip6: STRING, source: STRING,
f10kItem: STRING, chunkSeqId: INTEGER, text: STRING,
textEmbedding: LIST} Form {names: LIST, formId: STRING, cik:
STRING, cusip6: STRING, source: STRING} Company {names:
LIST, cusip6: STRING, companyName: STRING, cusip: STRING}
Manager {managerCik: STRING, managerName: STRING,
managerAddress: STRING} Relationship properties: SECTION
{f10kItem: STRING} OWNS_STOCK_IN {reportCalendarOrQuarter:
STRING, value: FLOAT, shares: INTEGER} The relationships:
(:Chunk)-[:NEXT]->(:Chunk) (:Chunk)-[:PART_OF]->(:Form)
(:Form)-[:SECTION]->(:Chunk) (:Company)-[:FILED]->(:Form)
(:Manager)-[:OWNS_STOCK_IN]->(:Company)


In [ ]:
cypher = """
    MATCH (chunk:Chunk)
    RETURN chunk.chunkId as chunkId LIMIT 1
    """

chunk_rows = kg.query(cypher)
print(chunk_rows)

[{'chunkId': '0000950170-23-027948-item1-chunk0000'}]


In [ ]:
chunk_first_row = chunk_rows[0]
print(chunk_first_row)

{'chunkId': '0000950170-23-027948-item1-chunk0000'}


In [ ]:
ref_chunk_id = chunk_first_row['chunkId']
ref_chunk_id

'0000950170-23-027948-item1-chunk0000'

In [ ]:
cypher = """
    MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f:Form)
    RETURN f.source
    """

kg.query(cypher, params={'chunkIdParam': ref_chunk_id})

[{'f.source': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}]

In [ ]:
cypher = """
MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f:Form),
    (com:Company)-[:FILED]->(f)
RETURN com.companyName as name
"""

kg.query(cypher, params={'chunkIdParam': ref_chunk_id})

[{'name': 'NETAPP INC'}]

In [ ]:
cypher = """
MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f:Form),
        (com:Company)-[:FILED]->(f),
        (mgr:Manager)-[:OWNS_STOCK_IN]->(com)
RETURN com.companyName,
        count(mgr.managerName) as numberOfinvestors
LIMIT 1
"""

kg.query(cypher, params={
    'chunkIdParam': ref_chunk_id
})

[{'com.companyName': 'NETAPP INC', 'numberOfinvestors': 561}]

In [ ]:
cypher = """
    MATCH (:Chunk {chunkId: $chunkIdParam})-[:PART_OF]->(f:Form),
        (com:Company)-[:FILED]->(f),
        (mgr:Manager)-[owns:OWNS_STOCK_IN]->(com)
    RETURN mgr.managerName + " owns " + owns.shares +
        " shares of " + com.companyName +
        " at a value of $" +
        apoc.number.format(toInteger(owns.value)) AS text
    LIMIT 10
    """
kg.query(cypher, params={
    'chunkIdParam': ref_chunk_id
})

[{'text': 'Royal Bank of Canada owns 842850 shares of NETAPP INC at a value of $64,395,000,000'},
 {'text': 'SHELTON CAPITAL MANAGEMENT owns 39124 shares of NETAPP INC at a value of $2,989,085,000'},
 {'text': 'PUBLIC EMPLOYEES RETIREMENT SYSTEM OF OHIO owns 106941 shares of NETAPP INC at a value of $8,170,000'},
 {'text': 'WILBANKS SMITH & THOMAS ASSET MANAGEMENT LLC owns 6617 shares of NETAPP INC at a value of $505,539,000'},
 {'text': 'DEPRINCE RACE & ZOLLO INC owns 320581 shares of NETAPP INC at a value of $24,492,389,000'},
 {'text': 'COMMERCE BANK owns 101422 shares of NETAPP INC at a value of $7,748,640,000'},
 {'text': 'D. E. Shaw & Co., Inc. owns 323440 shares of NETAPP INC at a value of $24,710,816,000'},
 {'text': 'EDMP, INC. owns 4698 shares of NETAPP INC at a value of $358,927,000'},
 {'text': 'NATIXIS ADVISORS, L.P. owns 40296 shares of NETAPP INC at a value of $3,079,000'},
 {'text': 'Smithfield Trust Co owns 215 shares of NETAPP INC at a value of $16,000'}]

In [ ]:
results = kg.query(cypher, params={
    'chunkIdParam': ref_chunk_id
})
print(textwrap.fill(results[0]['text'], 60))

Royal Bank of Canada owns 842850 shares of NETAPP INC at a
value of $64,395,000,000


In [ ]:
vector_store = Neo4jVector.from_existing_graph(
    embedding=GoogleGenerativeAIEmbeddings(
        model="models/text-embedding-004",
        google_api_key=GEMINI_API_KEY
    ),
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name=VECTOR_INDEX_NAME,
    node_label=VECTOR_NODE_LABEL,
    text_node_properties=[VECTOR_SOURCE_PROPERTY],
    embedding_node_property=VECTOR_EMBEDDING_PROPERTY,
)
# Create a retriever from the vector store
retriever = vector_store.as_retriever()

In [ ]:
# Create a chatbot Question & Answer chain from the retriever
plain_chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        google_api_key=GEMINI_API_KEY,
        temperature=0
    ),
    chain_type="stuff",
    retriever=retriever
)

In [ ]:
investment_retrieval_query = """
MATCH (node)-[:PART_OF]->(f:Form),
    (f)<-[:FILED]-(com:Company),
    (com)<-[owns:OWNS_STOCK_IN]-(mgr:Manager)
WITH node, score, mgr, owns, com
    ORDER BY owns.shares DESC LIMIT 10
WITH collect (
    mgr.managerName +
    " owns " + owns.shares +
    " shares in " + com.companyName +
    " at a value of $" +
    apoc.number.format(toInteger(owns.value)) + "."
) AS investment_statements, node, score
RETURN apoc.text.join(investment_statements, "\n") +
    "\n" + node.text AS text,
    score,
    {
      source: node.source
    } as metadata
"""

In [ ]:
vector_store_with_investment = Neo4jVector.from_existing_index(
    embedding=GoogleGenerativeAIEmbeddings(
        model="models/text-embedding-004",
        google_api_key=GEMINI_API_KEY
    ),
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database="neo4j",
    index_name=VECTOR_INDEX_NAME,
    text_node_property=VECTOR_SOURCE_PROPERTY,
    retrieval_query=investment_retrieval_query,
)

# Create a retriever from the vector store
retriever_with_investments = vector_store_with_investment.as_retriever()

In [ ]:
# Create a chatbot Question & Answer chain from the retriever
investment_chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        google_api_key=GEMINI_API_KEY,
        temperature=0
    ),
    chain_type="stuff",
    retriever=retriever_with_investments
)

In [ ]:
question = "In a single sentence, tell me about Netapp."

In [ ]:
plain_chain(
    {"question": question},
    return_only_outputs=True,
)

<ipython-input-43-7db4233d9967>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  plain_chain(


{'answer': 'NetApp is a global cloud-led, data-centric software company that helps customers manage applications and data across hybrid multicloud environments.\n',
 'sources': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}

In [ ]:
investment_chain(
    {"question": question},
    return_only_outputs=True,
)

{'answer': 'NetApp is a global cloud-led, data-centric software company that helps customers manage applications and data across hybrid multicloud environments.\n',
 'sources': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}

In [ ]:
question = "In a single sentence, tell me about Netapp investors."

In [ ]:
plain_chain(
    {"question": question},
    return_only_outputs=True,
)

{'answer': 'NetApp investors include global enterprises, local businesses, and government installations who seek to maximize the business value of their IT and cloud investments.\n',
 'sources': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}

In [ ]:
investment_chain(
    {"question": question},
    return_only_outputs=True,
)

{'answer': 'VANGUARD GROUP INC, BlackRock Inc., and PRIMECAP MANAGEMENT CO/CA/ are among the top investors in Netapp.\n',
 'sources': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}

In [ ]:
question = "Where is netapp located."

In [ ]:
plain_chain(
    {"question": question},
    return_only_outputs=True,
)

{'answer': 'NetApp is headquartered in San Jose, California.\n',
 'sources': 'https://www.sec.gov/Archives/edgar/data/1002047/000095017023027948/0000950170-23-027948-index.htm'}

In [ ]:
question = "What are other comapnies located in San Jose, California."

In [ ]:
plain_chain(
    {"question": question},
    return_only_outputs=True,
)

{'answer': "I don't know.", 'sources': ''}

In [ ]:
investment_chain(
    {"question": question},
    return_only_outputs=True,
)

{'answer': 'I am sorry, but I cannot answer the question. The provided documents only mention NetApp, Inc. which is headquartered in San Jose, California.',
 'sources': ''}

In [ ]:
question = "what extra inof do you have about netapp."

In [ ]:
plain_chain(
    {"question": question},
    return_only_outputs=True,
)

{'answer': "NetApp, Inc. is a global cloud-led, data-centric software company headquartered in San Jose, California. They offer cloud services and storage infrastructure powered by intelligent data management software, enabling applications to run faster, more reliably, and more securely at a lower cost. NetApp's offerings include:\n\n*   **BlueXP Sync:** Migrates data to the cloud securely and efficiently.\n*   **BlueXP Tiering:** Reduces storage costs and maximizes data storage.\n*   **NetApp Backup:** Delivers backup and restore capabilities for cloud and on-premises data.\n*   **BlueXP Compliance:** Provides data discovery, mapping, and classification for data privacy regulations.\n*   **BlueXP Cache:** Delivers fast and secure access to data for users globally.\n*   **NetApp Cloud Insights:** An infrastructure monitoring tool.\n*   **Spot by NetApp:** A platform for cloud operations.\n*   **Instaclustr:** A platform that provides fully managed open-source databases, pipelines, and